In [1]:
! pip install biopython
! pip install -q condacolab
import condacolab
condacolab.install()
! conda install -c bioconda seqkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.4 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - seqkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    

In [1]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# https://drive.google.com/file/d/1XZAwTCcIlANrd1wwR12_rbi1NvaQINos/view?usp=drive_link
# https://drive.google.com/file/d/1_dl0q_4JJe8IJxB1L2qDGc445gVlhWN-/view?usp=drive_link
# https://drive.google.com/file/d/14Ckpl7W6zfItYvDUmvm7lqbkd48CG20i/view?usp=drive_link
id_links = {"hw_file1.fasta":"1XZAwTCcIlANrd1wwR12_rbi1NvaQINos",
            "hw_file2.fasta":"1_dl0q_4JJe8IJxB1L2qDGc445gVlhWN-",
            "hw_file3.fasta":"14Ckpl7W6zfItYvDUmvm7lqbkd48CG20i"}
for name, file_id in id_links.items():
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(name)

In [6]:
import requests
import re
import json
from Bio import SeqIO
import subprocess
import sys
import argparse

In [29]:
class BioPythonLib:
    def __init__(self, file_name):
        self.regex = {"Protein": "^.*([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}).*$",
                      "DNA": "^.*(ENS[A-Z]{0,4}|MGP_[A-Za-z0-9]{0,10}_)(E|FM|G|GT|P|R|T)(\d{11}).*$"}
        self.filename = file_name
        self.SeqIO_ids = []
        self.output = {"seqkit_result": self.seqkit_stats()}
        self.biopython_parser()
        self.access_database()
        self.show_output(self.output)

    def seqkit_stats(self):
        seqkit = subprocess.run(("seqkit", "stats", self.filename, "-a"),
                                capture_output=True,
                                text=True)

        # If error arises
        if (seqkit.stderr != ''):
            print("Error:", seqkit.stderr)
            sys.exit(1)

        # Output
        if (seqkit.stdout == ''):
            print("Output is empty. Check the fasta file")
            sys.exit(1)

        seqkit_out = seqkit.stdout.strip().split('\n')

        # split names and values
        prop_names = seqkit_out[0].split()[1:]
        prop_vals = seqkit_out[1].split()[1:]

        seq_result = dict(zip(prop_names, prop_vals))
        # using zip
        self.seqkit_result = {"fasta_seqkit_stat_info": seq_result,
                              "fasta_type": seq_result['type'],
                              "fasta_num_seqs": int(seq_result['num_seqs']),
                              }

        return self.seqkit_result

    def biopython_parser(self):
        sequences = SeqIO.parse(self.filename, 'fasta')  # seq input; returns an iterator
        regex = self.regex[self.seqkit_result["fasta_type"]]

        for seq in sequences:

            seq_description = seq.description
            seq_sequence = seq.seq

            # seq_id
            match = re.fullmatch(regex, seq_description)

            if match:
                if self.seqkit_result["fasta_type"] == 'DNA':
                    id_chunks = re.findall(regex, seq_description)
                    seq_id = [''.join(chunk) for chunk in id_chunks][0]

                if self.seqkit_result["fasta_type"] == 'Protein':
                    seq_id = re.findall(regex, seq_description)[0][0]

                self.SeqIO_ids.append(seq_id)
                self.output[f'seq_id_{seq_id}_info'] = {"description": seq_description, "sequence": seq_sequence}

            else:
                print("No ID match found.")
                sys.exit(1)

    def get_uniprot(self, ids: list):
        accessions = ','.join(ids)
        endpoint = "https://rest.uniprot.org/uniprotkb/accessions"
        http_function = requests.get
        http_args = {'params': {'accessions': accessions}}

        return http_function(endpoint, **http_args)

    def get_ensembl(self, ids: list):
        server = "https://rest.ensembl.org"
        ext = "/lookup/id"
        headers = {"Content-Type": "application/json"}
        x = {"ids": ids}
        y = json.dumps(x)
        resp = requests.post(server + ext, headers=headers, data=y)
        return resp

    def parse_response_uniprot(self, resp: dict):
        resp = resp.json()
        resp = resp["results"]
        output = {}
        for val in resp:
            acc = val['primaryAccession']
            species = val['organism']['scientificName']
            gene = val['genes']
            seq = val['sequence']
            output[acc] = {'organism': species, 'geneInfo': gene, 'sequenceInfo': seq, 'type': 'protein'}

        return output

    def parse_response_ensembl(self, resp: dict):
        resp = resp.json()
        output = {}
        for val in resp.values():
            id = val['id']
            species = val['species']
            gene = {
                'assembly_name': val['assembly_name'],
                'biotype': val['biotype'],
                'canonical_transcript': val['canonical_transcript']
            }
            seq = {
                'seq_region_name': val['seq_region_name'],
                'start': val['start'],
                'end': val['end']
            }
            object_type = val['object_type']
            output[id] = {'organism': species,
                          'geneInfo': gene,
                          'sequenceInfo': seq,
                          'type': object_type}

        return output

    def access_database(self):
        ids = self.SeqIO_ids
        if self.seqkit_result["fasta_type"] == 'Protein':
            data = self.parse_response_uniprot(self.get_uniprot(ids))
            self.output["DB_name"] = "uniprot"
            self.output["DB_result"] = data
        elif self.seqkit_result["fasta_type"] == 'DNA':
            data = self.parse_response_ensembl(self.get_ensembl(ids))
            self.output["DB_name"] = "ENSEMBL"
            self.output["DB_result"] = data

    def show_output(self, v, indent=0):
        for key, value in v.items():
            print('\t' * indent + str(key))
            if isinstance(value, dict):
                self.show_output(value, indent + 1)
            else:
                print('\t' * (indent + 1) + str(value))

In [30]:
file_name = "hw_file2.fasta"
bio = BioPythonLib(file_name)

seqkit_result
	fasta_seqkit_stat_info
		format
			FASTA
		type
			DNA
		num_seqs
			6
		sum_len
			86
		min_len
			9
		avg_len
			14.3
		max_len
			23
		Q1
			10
		Q2
			13.5
		Q3
			17
		sum_gap
			0
		N50
			16
		N50_num
			3
		Q20(%)
			0
		Q30(%)
			0
		AvgQual
			0
		GC(%)
			45.35
	fasta_type
		DNA
	fasta_num_seqs
		6
seq_id_ENSMUSG00000096749_info
	description
		ENSMUST00000196221.2 cds chromosome:GRCm39:14:54350925:54350933:1 gene:ENSMUSG00000096749.3 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:Trdd1 description:T cell receptor delta diversity 1 [Source:MGI Symbol;Acc:MGI:4439547]
	sequence
		ATGGCATAT
seq_id_ENSMUSG00000096176_info
	description
		ENSMUST00000177564.2 cds chromosome:GRCm39:14:54359683:54359698:1 gene:ENSMUSG00000096176.2 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:Trdd2 description:T cell receptor delta diversity 2 [Source:MGI Symbol;Acc:MGI:4439546]
	sequence
		ATCGGAGGGATACGAG
seq_id_ENSG00000271336_info
	description
		